<a href="https://colab.research.google.com/github/sharonapa/dl_course/blob/main/ex3_deepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pretty_midi

In [ ]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from tensorboard import notebook
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import Average, Dense, Embedding, Flatten, Input, LSTM,Bidirectional,Dropout,Activation
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import Model, Sequential
import json
import numpy as np
import os
import pandas as pd
#import pretty_midi
import tensorflow as tf
import time

In [ ]:
# params
embd_size = 300
vocab_size = -1
seq_len = 5
batch_size = 8

In [ ]:
def read_data(file_name):
    songs_df = pd.read_csv(file_name, sep='\n', header=None)
    #split to 3 cols by the ',' sign
    songs_df_clean = songs_df.iloc[:, 0].str.rstrip(r'&, ').str.extract(r'([^,]+),([^,]+),(.+)')
    songs_df_clean.columns = ['artist', 'title', 'lyrics']
    print('number of songs = {}'.format(songs_df_clean.shape[0]))
    return songs_df_clean
file_name_train = '/content/lyrics_train_set.csv'
df_songs_train = read_data(file_name_train)
file_name_test = '/content/lyrics_test_set.csv'
df_songs_test = read_data(file_name_test)


number of songs = 600
number of songs = 5


In [ ]:
def clean_lyrics(df_songs):
    #df_songs['lyrics'] = df_songs.lyrics.str.replace('&', '\n') #in order to learn the break line
    df_songs['lyrics'] = df_songs.lyrics.str.lower()
    df_songs['lyrics_clean'] = df_songs.lyrics.str.replace('[,.?!)()]', '')
    return df_songs
df_songs_train = clean_lyrics(df_songs_train)


In [ ]:
# smaple data
df_songs_train.head()

,artist,title,lyrics,lyrics_clean
0,elton john,candle in the wind,goodbye norma jean & though i never knew you a...,goodbye norma jean & though i never knew you a...
1,gerry rafferty,baker street,winding your way down on baker street & lite i...,winding your way down on baker street & lite i...
2,gerry rafferty,right down the line,you know i need your love & you've got that ho...,you know i need your love & you've got that ho...
3,2 unlimited,tribal dance,come on check it out ya'll & (come on come on!...,come on check it out ya'll & come on come on &...
4,2 unlimited,let the beat control your body,let the beat control your body & let the beat ...,let the beat control your body & let the beat ...


In [ ]:
all_words = df_songs_train.lyrics_clean.str.cat()
corpus = all_words.split()
print('corpus size is {}'.format(len(corpus)))

corpus size is 182298


In [ ]:
emb_model = Word2Vec([corpus, ["UNK"]], min_count=1, size=embd_size)  # Handling unknown words
emb_model.save('./emb_model.bin')
print(emb_model)
vocab_size = len(emb_model.wv.vocab)

Word2Vec(vocab=8435, size=300, alpha=0.025)


In [ ]:
# prepare seq and next word
def prepare_sequences(corpus):
    sequences = []
    next_words = []
    len(corpus)
    for i in range(len(corpus)):
        if seq_len+i < len(corpus):
            seq = corpus[i:seq_len+i]
            sequences.append(seq)
            next_word = corpus[seq_len+i]
            next_words.append(next_word)
    return sequences,next_words
sequences_train,next_words_train = prepare_sequences(corpus)



In [ ]:
# turn all seqs to word2vec embdeing
sequences_train_wv = [] 
for seq in sequences_train:
    seq_wv = []
    for word in seq:
        seq_wv.append(emb_model[word])
    sequences_train_wv.append(seq_wv)
sequences_train_wv_np = np.array(sequences_train_wv)
next_words_train_np = np.array(next_words_train)
# sequences_train_wv = [for x in sequences_train]
# sequences_train_np = np.array(sequences_train)
# sequences_train_np = sequences_train_np
# sequences_train_np.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [ ]:
# get the index of rach word from word2vec
# prepare list of lists with idx
 
emb_model.wv.index2word[0]
word2index = {word:idx for idx,word in enumerate(emb_model.wv.index2word)}
word2index['&']

In [ ]:
emb_mat = emb_model.wv.syn0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [ ]:
def get_model(vocab_size, embedding_dim, rnn_units, batch_size,dropout = 0.2):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=seq_len, weights=[emb_mat]))
    #make trainable false

    model.add(LSTM(rnn_units, return_sequences=True, stateful=False, recurrent_initializer='glorot_uniform'))
    model.add(Dense(rnn_units,activation='relu'))
    model.add(Dense(vocab_size,activation='softmax'))
    # model = Sequential()
    # model.add(LSTM(128, input_shape=(embedding_dim, seq_len)))
    # # model.add(Bidirectional(LSTM(rnn_units),input_shape=(batch_size, seq_len, embedding_dim)))
    # if dropout > 0:
    #     model.add(Dropout(dropout))
    # model.add(Dense(vocab_size))
    # model.add(Activation('softmax'))

    return model

In [ ]:
lstm_model = get_model(vocab_size,embd_size,10, batch_size)
lstm_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 5, 300)            2530500   
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 10)             12440     
_________________________________________________________________
dense (Dense)                (None, 5, 10)             110       
_________________________________________________________________
dense_1 (Dense)              (None, 5, 8435)           92785     
Total params: 2,635,835
Trainable params: 2,635,835
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# sparse_categorical_crossentropy : we can use array of index
# make loss : categorical_crossentropy : we need to do categorical array with to_catgorical of keras...


In [ ]:
 x_train=sequences_train_wv_np.reshape(sequences_train_wv_np.shape[0],sequences_train_wv_np.shape[2],5)


In [ ]:
next_words_train_np[]

'knew'

In [ ]:

lstm_model.fit(x=x_train,y=next_words_train_np,batch_size=batch_size)

UnimplementedError: ignored